In [1]:

import time
import requests
import json
import pandas as pd
import config
import matplotlib.pyplot as plt
import datetime
from csv import writer


In [5]:
       
def gen_point_data(name, gas):

    #normalize data
    df = pd.json_normalize(data[name])

    # List all records
    ls = df['list'][0]
    df_size = len(ls)
    
    #retrieve gas and make list of daily mean values
    compoent_list = []
    compoent_count = 0
    for i in range(df_size):
        compoent_count+=ls[i]['components'][gas]
        # Average for each day
        if (i%24 == 0):   
            compoent_daily = round(compoent_count/24, 5)
            compoent_list.append(compoent_daily)
            compoent_count = 0
    return compoent_list

#retrieve city, lat, lon 
city_df = pd.read_csv('C:\\Users\\Kurly\\Downloads\\Universal-Embeddings-testing\\Universal-Embeddings-Nick\\data\\city_lat_lon.csv')
city_count=len(city_df)

#list of gases in data
gases= ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
gas_lenght= len(gases)

#create colums for dataframe with time conversion from unix to human-readable
for i in range(gas_lenght):   
    #first and last hour
    T_START=1606456800
    T_END=1623128400
    num_entries = int((T_END - T_START) / 86400)
    time_step = T_START
    with open('C:\\Users\\Kurly\\Downloads\\Universal-Embeddings-testing\\Universal-Embeddings-Nick\\data\\'+gases[i]+'_data.csv', 'w', newline='') as f:
        fd=writer(f)
        col_names = ['city', 'lat', 'lon']
        for j in range(num_entries):
            # Convert daily interval to human-readable
            timedate = datetime.datetime.fromtimestamp(time_step)
            time_string = timedate.strftime(gases[i]+'_%Y_%m_%d')
            # Increment time_step
            time_step+=86400
            # Append col to list\
            col_names.append(time_string)
        fd.writerow(col_names)
    f.close()

#create cvs dataframe for every gas type    
for index in range(6):
    file_i = index + 1
    
    #load json txt file to variable
    with open('raw_data_'+str(file_i)+'_6.txt') as raw_data:
        print("loading data")
        data = json.load(raw_data)
    raw_data.close()
    
    #load city info for key, lat/lon for correcty city confirmation, and create new df csv file
    #print which json file is going to be worked on
    print(index, "Start")
    data_key = data.keys()
        for i in range(city_count): 
            count=0
            data_key_lenght=len(data_key)
            city_name = city_df.iloc[i][0]
            city_lat = city_df.iloc[i][1]
            city_lon = city_df.iloc[i][2]
            city_info=[city_name, city_lat, city_lon]
            # Retrieve particulate list; write row to csv
            if city_name in data_key:
                #try function avoid error with cities that were not loaded correctly or city had no info from API
                try:
                    df = pd.json_normalize(data[city_name])
                    lon = df['coord.lon'][0]
                    #indure city and lon values are the same to insure correct city
                    if city_lon==str(lon):
                        for j in range(gas_lenght):
                            city_info=[city_name, city_lat, city_lon]
                            entry = gen_point_data(city_name, gases[j])
                            city_info+=entry
                            #write to df to csv file
                            with open('C:\\Users\\Kurly\\Downloads\\Universal-Embeddings-testing\\Universal-Embeddings-Nick\\data\\'+gases[j]+'_data.csv', 'a', newline='') as f:
                                fd=writer(f)
                                fd.writerow(city_info)
                            f.close()
                # do nothing and continue with next city if error pops up
                except KeyError:
                    pass
            #print a value to see progression of loaded json file
            if(i%1000==0):
                print(i)
        #delete data variable for better memory management and print when json file is done
        del data                
        print(index, 'done')


Start
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
done
